In [33]:
import torch
import glob
import os
import pandas as pd
from tqdm import tqdm

In [34]:
a = torch.zeros((1,11,512,512))
b = torch.zeros((1,8,512,512))

ab = torch.cat([a,b],dim=1)
ab.size() == torch.Size([1,19,512,512])

True

In [35]:
data_root = 'out'
path_list = ['psa_80k','hr48','ocr_net','swin']
w_list = [
    [0.244,0.249,0.179,0.193,0.279,0.299,0.430,0.336,0.230,0.479,0.109],
	[0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.240],
	[0.345,0.383,0.356,0.292,0.243,0.347,0.188,0.310,0.395,0.056,0.000],
	[0.411,0.368,0.464,0.516,0.477,0.353,0.383,0.354,0.375,0.465,0.651],
]

# w_list = [[1 for _ in range(11)] for _ in range(len(path_list))] # if don't want to use weight

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
pt_lists = []
for p in path_list:
    pt_list = sorted(glob.glob(f'{data_root}/{p}/*.pt'))
    pt_lists.append(pt_list)

os.makedirs(f'{data_root}/essamble', exist_ok=True)

b = torch.zeros((1,8,512,512)).to(device)

for p in tqdm(zip(*pt_lists),total=len(pt_list)):
    logits = list(map(lambda x: torch.load(x),p))
    for l,w in zip(logits,w_list):
        for l_,w_ in zip(l[0],w):
            l_.data *= torch.tensor(w_).to(device)
        if not l.size() == torch.Size([1,19,512,512]):
            l.data = torch.cat([l.data,b],dim=1)

    logit_sum = sum(logits)
    result = torch.argmax(logit_sum,dim=1)
    file_name = p[0].split('/')[-1].replace('.pt','.txt')
    with open(f'{data_root}/essamble/{file_name}', 'w') as f:
        for r in result[0]:
            r_list = r.tolist()
            f.write(' '.join(map(str,r_list)) + '\n')

100%|██████████| 624/624 [01:29<00:00,  6.99it/s]


In [36]:
print('\nmaking submission ...')
dic = {"image_id":[],"PredictionString":[]}
for path in tqdm(sorted(glob.glob(f'{data_root}/essamble/*.txt'))):
    image_id = path.replace(f'{data_root}/essamble/','').replace('_03_','_03/').replace('vt_','vt/').replace('.txt','.jpg')
    with open(path, 'r') as f:
        value_list = f.readlines()
        new_value_list = []
        for i in range(0,len(value_list),2):
            value = value_list[i].split(' ')
            value = ' '.join(value[0::2])
            new_value_list.append(value)

    dic["image_id"].append(image_id)
    dic["PredictionString"].append(' '.join(new_value_list))

df = pd.DataFrame(dic)
df.to_csv(f'{data_root}/essamble/_submission.csv',index=False)

  2%|▏         | 14/624 [00:00<00:04, 138.66it/s]


making submission ...


100%|██████████| 624/624 [00:05<00:00, 123.76it/s]
